In [7]:
import numpy as np 
import pandas as pd 

from tqdm import tqdm 
import time, datetime as dt

In [8]:
def Modified_Treasury_Yield_df_to_Month(df,yield_column_name):

    df = df[df[str(yield_column_name)]!='.']
    df[str(yield_column_name)] = df[str(yield_column_name)].astype(float)
    df = df.fillna('pad')

    df.index = pd.DatetimeIndex(df.DATE)
    df = df.to_period('M')
    df = df.drop(['DATE'],axis=1)
    df = df.groupby('DATE')[str(yield_column_name)].agg('median').reset_index(name=str(yield_column_name))
    
    return df

df_Fred_2_Year_Treasury_Yield = pd.read_csv(r'./FRED 2 year treasury yield.csv')
df_Fred_10_Year_Treasury_Yield = pd.read_csv(r'./FRED 10 year treasury yield.csv')


df_Fred_2_Year_Treasury_Yield = Modified_Treasury_Yield_df_to_Month(df=df_Fred_2_Year_Treasury_Yield ,yield_column_name='DGS2')
df_Fred_10_Year_Treasury_Yield = Modified_Treasury_Yield_df_to_Month(df=df_Fred_10_Year_Treasury_Yield ,yield_column_name='DGS10') 
df_FRED_Treasury_Yield = pd.merge(df_Fred_2_Year_Treasury_Yield,df_Fred_10_Year_Treasury_Yield)
df_FRED_Treasury_Yield.index = df_FRED_Treasury_Yield['DATE']
df_FRED_Treasury_Yield = df_FRED_Treasury_Yield.drop(['DATE'],axis=1)
df_FRED_Treasury_Yield

/var/folders/63/4xwgsk7j6gg3kmjr9bt6jn5m0000gn/T/ipykernel_55340/3988143676.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(yield_column_name)] = df[str(yield_column_name)].astype(float)


,DGS2,DGS10
DATE,,
2000-01,6.445,6.680
2000-02,6.630,6.560
2000-03,6.520,6.260
2000-04,6.380,5.990
2000-05,6.825,6.455
...,...,...
2022-05,2.620,2.890
2022-06,3.060,3.130
2022-07,3.030,2.920


In [9]:
df_Fred_CPI = pd.read_csv(r'./FRED CPI.csv')
df_Fred_PPI = pd.read_csv(r'./FRED PPI.csv')

df_FRED_Price_Index = pd.merge(df_Fred_CPI,df_Fred_PPI)
df_FRED_Price_Index.index = pd.DatetimeIndex(df_FRED_Price_Index.DATE)
df_FRED_Price_Index = df_FRED_Price_Index.drop(['DATE'],axis=1)
df_FRED_Price_Index = df_FRED_Price_Index.to_period('M')
df_FRED_Price_Index

,CPIAUCSL_PC1,CPILFESL_PC1,PPIACO
DATE,,,
1964-01,1.64258,2.22222,31.800
1964-02,1.41076,1.89873,31.600
1964-03,1.40937,1.57729,31.600
1964-04,1.54199,1.57729,31.600
1964-05,1.54048,1.57729,31.500
...,...,...,...
2022-05,8.51641,6.01128,273.251
2022-06,8.99522,5.91245,280.204
2022-07,8.48213,5.91153,272.328


In [10]:
df_SP500_Investment_Grade_Index = pd.read_excel(r'./S&P 500 Investment Bond index.xls',skiprows=5)
df_SP500_High_Yield_Bond_Index = pd.read_excel(r'./S&P 500 High-Yield Bond Index.xls',skiprows=5)

df_SP500_High_Yield_Bond_Index.columns = df_SP500_High_Yield_Bond_Index.iloc[0]
df_SP500_High_Yield_Bond_Index = df_SP500_High_Yield_Bond_Index[1:-4]

df_SP500_Investment_Grade_Index.columns = df_SP500_Investment_Grade_Index.iloc[0]
df_SP500_Investment_Grade_Index = df_SP500_Investment_Grade_Index[1:-4]

df_SP500_Investment_Grade_Index.columns = ['DATE','S&P 500 Bond Index']
df_SP500_Investment_Grade_Index.index = pd.DatetimeIndex(df_SP500_Investment_Grade_Index['DATE'])
df_SP500_Investment_Grade_Index = df_SP500_Investment_Grade_Index.drop(['DATE'],axis=1)
df_SP500_Investment_Grade_Index

df_SP500_High_Yield_Bond_Index.columns = ['DATE','S&P U.S. High Yield Corporate Bond Index']
df_SP500_High_Yield_Bond_Index.index = pd.DatetimeIndex(df_SP500_High_Yield_Bond_Index['DATE'])
df_SP500_High_Yield_Bond_Index = df_SP500_High_Yield_Bond_Index.drop(['DATE'],axis=1)


df_SP500_Bond_Index = pd.merge(df_SP500_High_Yield_Bond_Index,df_SP500_Investment_Grade_Index,left_index=True,right_index=True)
df_SP500_Bond_Index = df_SP500_Bond_Index.to_period('M').groupby('DATE')['S&P 500 Bond Index','S&P U.S. High Yield Corporate Bond Index'].agg('mean')
df_SP500_Bond_Index 

/var/folders/63/4xwgsk7j6gg3kmjr9bt6jn5m0000gn/T/ipykernel_55340/4269684182.py:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_SP500_Bond_Index = df_SP500_Bond_Index.to_period('M').groupby('DATE')['S&P 500 Bond Index','S&P U.S. High Yield Corporate Bond Index'].agg('mean')


,S&P 500 Bond Index,S&P U.S. High Yield Corporate Bond Index
DATE,,
2012-09,366.200000,454.770000
2012-10,368.197391,458.133043
2012-11,369.553636,458.723182
2012-12,369.263810,466.750476
2013-01,368.628261,473.729565
...,...,...
2022-06,464.605714,681.532381
2022-07,469.187500,678.044500
2022-08,473.600435,699.186522


In [11]:
Macro_Factor_df = pd.merge(df_FRED_Price_Index,df_FRED_Treasury_Yield,left_index=True,right_index=True)
Macro_Factor_df

,CPIAUCSL_PC1,CPILFESL_PC1,PPIACO,DGS2,DGS10
DATE,,,,,
2000-01,2.79296,2.10706,128.300,6.445,6.680
2000-02,3.21797,2.16401,129.800,6.630,6.560
2000-03,3.76214,2.44735,130.800,6.520,6.260
2000-04,3.01386,2.26886,130.700,6.380,5.990
2000-05,3.13253,2.37960,131.600,6.825,6.455
...,...,...,...,...,...
2022-05,8.51641,6.01128,273.251,2.620,2.890
2022-06,8.99522,5.91245,280.204,3.060,3.130
2022-07,8.48213,5.91153,272.328,3.030,2.920


In [12]:
Macro_Factor_df.columns = ['CPI YoY','Core CPI YoY','PPI index(All Commodity)','2-Year Treasury Yield','10-Year Treasury Yield']
Macro_Factor_df

,CPI YoY,Core CPI YoY,PPI index(All Commodity),2-Year Treasury Yield,10-Year Treasury Yield
DATE,,,,,
2000-01,2.79296,2.10706,128.300,6.445,6.680
2000-02,3.21797,2.16401,129.800,6.630,6.560
2000-03,3.76214,2.44735,130.800,6.520,6.260
2000-04,3.01386,2.26886,130.700,6.380,5.990
2000-05,3.13253,2.37960,131.600,6.825,6.455
...,...,...,...,...,...
2022-05,8.51641,6.01128,273.251,2.620,2.890
2022-06,8.99522,5.91245,280.204,3.060,3.130
2022-07,8.48213,5.91153,272.328,3.030,2.920


In [13]:
Macro_Factor_df['Credit Spread(10 Year and 2 Year Treasury)'] = Macro_Factor_df['10-Year Treasury Yield'] - Macro_Factor_df['2-Year Treasury Yield']
Macro_Factor_df

,CPI YoY,Core CPI YoY,PPI index(All Commodity),2-Year Treasury Yield,10-Year Treasury Yield,Credit Spread(10 Year and 2 Year Treasury)
DATE,,,,,,
2000-01,2.79296,2.10706,128.300,6.445,6.680,0.235
2000-02,3.21797,2.16401,129.800,6.630,6.560,-0.070
2000-03,3.76214,2.44735,130.800,6.520,6.260,-0.260
2000-04,3.01386,2.26886,130.700,6.380,5.990,-0.390
2000-05,3.13253,2.37960,131.600,6.825,6.455,-0.370
...,...,...,...,...,...,...
2022-05,8.51641,6.01128,273.251,2.620,2.890,0.270
2022-06,8.99522,5.91245,280.204,3.060,3.130,0.070
2022-07,8.48213,5.91153,272.328,3.030,2.920,-0.110


In [14]:
Fama_French_3_Factor = pd.read_csv(r'./F-F_Research_Data_Factors.CSV')

Fama_French_3_Factor['DATE'] = Fama_French_3_Factor.apply(lambda x: str(x['DATE'])[:4] +"-" + str(x['DATE'])[4:6],axis=1)
Fama_French_3_Factor.index = pd.DatetimeIndex(Fama_French_3_Factor['DATE'])
Fama_French_3_Factor = Fama_French_3_Factor.drop(['DATE'],axis=1)
Fama_French_3_Factor = Fama_French_3_Factor.to_period('M')
Fama_French_3_Factor 

,Mkt-RF,SMB,HML,RF
DATE,,,,
1926-07,2.96,-2.56,-2.43,0.22
1926-08,2.64,-1.17,3.82,0.25
1926-09,0.36,-1.40,0.13,0.23
1926-10,-3.24,-0.09,0.70,0.32
1926-11,2.53,-0.10,-0.51,0.31
...,...,...,...,...
2022-04,-9.46,-1.41,6.19,0.01
2022-05,-0.34,-1.85,8.41,0.03
2022-06,-8.43,2.09,-5.97,0.06


In [15]:
Macro_Factor_df = pd.merge(Macro_Factor_df,Fama_French_3_Factor,left_index=True,right_index=True)
Macro_Factor_df

,CPI YoY,Core CPI YoY,PPI index(All Commodity),2-Year Treasury Yield,10-Year Treasury Yield,Credit Spread(10 Year and 2 Year Treasury),Mkt-RF,SMB,HML,RF
DATE,,,,,,,,,,
2000-01,2.79296,2.10706,128.300,6.445,6.680,0.235,-4.74,5.79,-1.91,0.41
2000-02,3.21797,2.16401,129.800,6.630,6.560,-0.070,2.45,21.42,-9.70,0.43
2000-03,3.76214,2.44735,130.800,6.520,6.260,-0.260,5.20,-17.23,8.17,0.47
2000-04,3.01386,2.26886,130.700,6.380,5.990,-0.390,-6.40,-6.68,7.26,0.46
2000-05,3.13253,2.37960,131.600,6.825,6.455,-0.370,-4.42,-6.09,4.81,0.50
...,...,...,...,...,...,...,...,...,...,...
2022-04,8.22414,6.13369,265.310,2.520,2.800,0.280,-9.46,-1.41,6.19,0.01
2022-05,8.51641,6.01128,273.251,2.620,2.890,0.270,-0.34,-1.85,8.41,0.03
2022-06,8.99522,5.91245,280.204,3.060,3.130,0.070,-8.43,2.09,-5.97,0.06
